In [ ]:
%pip install snowflake-connector-python

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import datetime
import os

# Leer variables de entorno
SNOWFLAKE_ACCOUNT = os.getenv('SNOWFLAKE_ACCOUNT')
SNOWFLAKE_USER = os.getenv('SNOWFLAKE_USER')
SNOWFLAKE_PASSWORD = os.getenv('SNOWFLAKE_PASSWORD')
SNOWFLAKE_DATABASE = os.getenv('SNOWFLAKE_DATABASE')
SNOWFLAKE_SCHEMA_ANALYTICS = os.getenv('SNOWFLAKE_SCHEMA_ANALYTICS', 'ANALYTICS')
SNOWFLAKE_WAREHOUSE = os.getenv('SNOWFLAKE_WAREHOUSE', 'COMPUTE_WH')

In [ ]:
# Instalación de dependencias y descarga de JARs para Spark-Snowflake
!pip install snowflake-snowpark-python snowflake-connector-python

# Crear directorio para JARs si no existe
import os
jars_dir = '/content/jars'
os.makedirs(jars_dir, exist_ok=True)

# Descargar JARs necesarios para Spark 3.x con Scala 2.12
# Snowflake Spark Connector compatible con Spark 3.x y Scala 2.12
snowflake_jar_url = "https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.12.0-spark_3.4/spark-snowflake_2.12-2.12.0-spark_3.4.jar"
snowflake_jdbc_url = "https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.14.4/snowflake-jdbc-3.14.4.jar"

# Descargar JARs
!wget -O {jars_dir}/spark-snowflake_2.12-2.12.0-spark_3.4.jar {snowflake_jar_url}
!wget -O {jars_dir}/snowflake-jdbc-3.14.4.jar {snowflake_jdbc_url}

print("JARs descargados exitosamente")

In [3]:
!mkdir -p /content/jars
!wget -q -O /content/jars/snowflake-jdbc-3.14.4.jar https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.14.4/snowflake-jdbc-3.14.4.jar
!wget -q -O /content/jars/spark-snowflake_2.12-2.12.0-spark_3.4.jar https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.12.0-spark_3.4/spark-snowflake_2.12-2.12.0-spark_3.4.jar
!ls -lh /content/jars


total 69M
-rw-r--r-- 1 root root  68M Dec  7  2023 snowflake-jdbc-3.14.4.jar
-rw-r--r-- 1 root root 755K May 23  2023 spark-snowflake_2.12-2.12.0-spark_3.4.jar


In [6]:
# Importar PySpark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import random
from pyspark.sql import types as T

from datetime import datetime, timedelta


# Configuración de JARs para Snowflake
jars_dir = '/content/jars'
spark_jars = f"{jars_dir}/spark-snowflake_2.12-2.12.0-spark_3.4.jar,{jars_dir}/snowflake-jdbc-3.14.4.jar"

# Crear sesión Spark simple
print("Creando sesión Spark...")

spark = SparkSession.builder \
    .appName("NYC_Taxi_OBT_Analysis") \
    .config("spark.jars", "/content/jars/snowflake-jdbc-3.14.4.jar,/content/jars/spark-snowflake_2.12-2.12.0-spark_3.4.jar") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print(f" Spark iniciado correctamente. Versión: {spark.version}")


test_count = spark.range(5).count()
print(f"Spark funcionando correctamente - Test: {test_count} registros")
print(f"Versión Spark: {spark.version}")

# Mostrar configuración activa
print(f"JARs configurados: {len(spark_jars.split(','))} archivos")

Creando sesión Spark...
 Spark iniciado correctamente. Versión: 3.5.1
Spark funcionando correctamente - Test: 5 registros
Versión Spark: 3.5.1
JARs configurados: 2 archivos


In [ ]:
sfOptions = {
    "sfURL": SNOWFLAKE_ACCOUNT,
    "sfUser": SNOWFLAKE_USER,
    "sfPassword": SNOWFLAKE_PASSWORD,
    "sfDatabase": SNOWFLAKE_DATABASE,
    "sfSchema": SNOWFLAKE_SCHEMA_ANALYTICS,
    "sfWarehouse": SNOWFLAKE_WAREHOUSE,
}

df = (
    spark.read
        .format("net.snowflake.spark.snowflake")
        .options(**sfOptions)
        .option("dbtable", "OBT_TRIPS")
        .load()
)

df.show(5, truncate=False)


+---------------------------------------+---------------------+-------------------+-------------------+-----------+-----------+------------+------------+-----------+----------------+-----+----------+----+-------+----------+--------------+-------------------+----------+---------------+--------------+-----------------------------+----------+---------------+------------+---------+---------------------------------+------------+--------------+------------+-----------------+---------+--------------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+--------------------+-----------+------------------+------------+-----------------+-------------+-------+----------------+---------------+---------------+------------+-------------+---------------+-------------+------------------------------------+--------------------------+--------------+-----------+------------+-----------------------------------------------------------+------

In [ ]:
# CONFIGURACIÓN INICIAL PARA VALIDACIONES
print("=== VALIDACIONES Y EXPLORACIÓN - OBT NYC TAXI ===")

# Conexión a Snowflake (reutilizar configuración del notebook anterior)
try:
    import snowflake.connector
    import time
    from datetime import datetime

    print("snowflake-connector-python disponible")

    # Configuración de conexión

    conn_config = {
        'user': SNOWFLAKE_USER,
        'password': SNOWFLAKE_PASSWORD,
        'account': SNOWFLAKE_ACCOUNT,
        'warehouse': SNOWFLAKE_WAREHOUSE,
        'database': SNOWFLAKE_DATABASE,
        'schema': SNOWFLAKE_SCHEMA_ANALYTICS
    }
    
    def execute_snowflake_sql(sql_query, fetch_results=True):
        """
        Ejecuta SQL directamente en Snowflake con resultados
        """
        try:
            conn = snowflake.connector.connect(**conn_config)
            cursor = conn.cursor()

            cursor.execute(sql_query)

            if fetch_results:
                results = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]
                return results, columns
            else:
                rows_affected = cursor.rowcount
                print(f" SQL ejecutado. Filas afectadas: {rows_affected:,}")
                return rows_affected, []

        except Exception as e:
            print(f"Error ejecutando SQL: {e}")
            return None, []
        finally:
            try:
                cursor.close()
                conn.close()
            except:
                pass

except ImportError:
    print("snowflake-connector-python no instalado")
    print("Instalar con: pip install snowflake-connector-python")

    def execute_snowflake_sql(sql_query, fetch_results=True):
        print("Connector no disponible. SQL para ejecutar manualmente:")
        print("-" * 60)
        print(sql_query)
        print("-" * 60)
        return None, []

print(" Configuración lista para preguntas de negocio en analytics.obt_trips")

=== VALIDACIONES Y EXPLORACIÓN - OBT NYC TAXI ===
snowflake-connector-python disponible
 Configuración lista para preguntas de negocio en analytics.obt_trips


## Debido al tiempo que tomaban en ejecutarse no se pudo ejecutar todo con spark, sin embargo se añadieron las consultas utilizando el conector de snowflake con python.

In [8]:
# A. TOP 10 ZONAS PICKUP POR VOLUMEN MENSUAL
print("ANÁLISIS A: Top 10 Zonas Pickup por Volumen Mensual")

# Top 10 zonas de pickup por volumen mensual
top_pickup_monthly = (
    df.groupBy("month", "pu_zone", "pu_borough")
    .agg(F.count("*").alias("num_trips"))
    .orderBy("month", F.col("num_trips").desc())
)

print("TOP 10 ZONAS PICKUP POR VOLUMEN MENSUAL:")
top_pickup_monthly.show(10)

ANÁLISIS A: Top 10 Zonas Pickup por Volumen Mensual
TOP 10 ZONAS PICKUP POR VOLUMEN MENSUAL:
+-----+--------------------+----------+---------+
|month|             pu_zone|pu_borough|num_trips|
+-----+--------------------+----------+---------+
|    1|Upper East Side S...| Manhattan|  2881290|
|    1|Upper East Side N...| Manhattan|  2765705|
|    1|      Midtown Center| Manhattan|  2715256|
|    1|        Midtown East| Manhattan|  2358939|
|    1|Times Sq/Theatre ...| Manhattan|  2334095|
|    1|Penn Station/Madi...| Manhattan|  2315438|
|    1|            Union Sq| Manhattan|  2172796|
|    1|         Murray Hill| Manhattan|  2150167|
|    1|        Clinton East| Manhattan|  2083432|
|    1| Lincoln Square East| Manhattan|  2061627|
+-----+--------------------+----------+---------+
only showing top 10 rows



In [15]:
# A. TOP 10 ZONAS PICKUP POR VOLUMEN MENSUAL - VERSIÓN SNOWFLAKE CONNECTOR
print("ANÁLISIS A: Top 10 Zonas Pickup por Volumen Mensual")

# Consulta SQL directa usando conector de Snowflake
sql_pickup_zones = """
SELECT
    month,
    pu_zone,
    pu_borough,
    COUNT(*) as num_trips
FROM analytics.obt_trips
WHERE pu_zone IS NOT NULL
GROUP BY month, pu_zone, pu_borough
ORDER BY month, COUNT(*) DESC
LIMIT 10
"""
execute_snowflake_sql(sql_pickup_zones)

ANÁLISIS A: Top 10 Zonas Pickup por Volumen Mensual


([(1, 'Upper East Side South', 'Manhattan', 2881290),
  (1, 'Upper East Side North', 'Manhattan', 2765705),
  (1, 'Midtown Center', 'Manhattan', 2715256),
  (1, 'Midtown East', 'Manhattan', 2358939),
  (1, 'Times Sq/Theatre District', 'Manhattan', 2334095),
  (1, 'Penn Station/Madison Sq West', 'Manhattan', 2315438),
  (1, 'Union Sq', 'Manhattan', 2172796),
  (1, 'Murray Hill', 'Manhattan', 2150167),
  (1, 'Clinton East', 'Manhattan', 2083432),
  (1, 'Lincoln Square East', 'Manhattan', 2061627)],
 ['MONTH', 'PU_ZONE', 'PU_BOROUGH', 'NUM_TRIPS'])

In [ ]:
# B. TOP 10 ZONAS DROPOFF POR VOLUMEN MENSUAL
print("ANÁLISIS B: Top 10 Zonas Dropoff por Volumen Mensual")

# Top 10 zonas de dropoff por volumen mensual
top_dropoff_monthly = (
    df.groupBy("month", "do_zone", "do_borough")
    .agg(F.count("*").alias("num_trips"))
    .orderBy("month", F.col("num_trips").desc())
)

print("TOP 10 ZONAS DROPOFF POR VOLUMEN MENSUAL:")
top_dropoff_monthly.show(10)

In [16]:
# B. TOP 10 ZONAS DROPOFF POR VOLUMEN MENSUAL
print("ANÁLISIS B: Top 10 Zonas Dropoff por Volumen Mensual")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    month,
    do_zone,
    do_borough,
    COUNT(*) as num_trips
FROM analytics.obt_trips
WHERE do_zone IS NOT NULL
GROUP BY month, do_zone, do_borough
ORDER BY month ASC, COUNT(*) DESC
limit 10
"""
execute_snowflake_sql(query)

ANÁLISIS B: Top 10 Zonas Dropoff por Volumen Mensual


([(1, 'Upper East Side North', 'Manhattan', 2914017),
  (1, 'Upper East Side South', 'Manhattan', 2582757),
  (1, 'Midtown Center', 'Manhattan', 2575709),
  (1, 'Murray Hill', 'Manhattan', 2175364),
  (1, 'Times Sq/Theatre District', 'Manhattan', 2071310),
  (1, 'Midtown East', 'Manhattan', 2050840),
  (1, 'Lincoln Square East', 'Manhattan', 1891516),
  (1, 'Union Sq', 'Manhattan', 1888098),
  (1, 'Clinton East', 'Manhattan', 1862376),
  (1, 'Upper West Side South', 'Manhattan', 1826451)],
 ['MONTH', 'DO_ZONE', 'DO_BOROUGH', 'NUM_TRIPS'])

In [ ]:
# C. EVOLUCIÓN MENSUAL DE TOTAL_AMOUNT Y TIP_PCT POR BOROUGH
print("ANÁLISIS C: Evolución Mensual de Total_Amount y Tip_Pct por Borough")

# Evolución mensual de total_amount y tip_pct por borough
evolution_monthly = (
    df.groupBy("year", "month", "pu_borough")
    .agg(F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.round(F.avg("tip_pct"), 2).alias("avg_tip_pct"))
    .orderBy("year", "month", "pu_borough")
)

print("EVOLUCIÓN MENSUAL DE TOTAL_AMOUNT Y TIP_PCT POR BOROUGH:")
evolution_monthly.show(10)

In [17]:
# C. EVOLUCIÓN MENSUAL DE TOTAL_AMOUNT Y TIP_PCT POR BOROUGH
print("ANÁLISIS C: Evolución Mensual de Total_Amount y Tip_Pct por Borough")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    year,
    month,
    pu_borough,
    ROUND(AVG(total_amount), 2) AS avg_total_amount,
    ROUND(AVG(tip_pct), 2) AS avg_tip_pct
FROM analytics.obt_trips
WHERE pu_borough IS NOT NULL
GROUP BY year, month, pu_borough
ORDER BY year, month, pu_borough;
"""
execute_snowflake_sql(query)

ANÁLISIS C: Evolución Mensual de Total_Amount y Tip_Pct por Borough


([(2001, 1, 'Brooklyn', 8.8, 0.0),
  (2001, 1, 'Manhattan', 19.88, 2.08),
  (2001, 1, 'Queens', 49.9, 4.7),
  (2001, 2, 'Queens', 3.8, 0.0),
  (2001, 8, 'Queens', 24.55, 0.0),
  (2002, 2, 'Manhattan', 17.9, 0.0),
  (2002, 10, 'Brooklyn', 28.59, 0.0),
  (2002, 10, 'Manhattan', 18.33, 8.62),
  (2002, 10, 'Queens', 51.39, 8.69),
  (2002, 10, 'Unknown', 21.15, 3.37),
  (2002, 12, 'Brooklyn', 31.3, 0.0),
  (2002, 12, 'Manhattan', 19.39, 3.59),
  (2002, 12, 'Queens', 48.09, 10.58),
  (2002, 12, 'Unknown', 6.03, 11.46),
  (2003, 1, 'Manhattan', 28.6, 4.39),
  (2003, 1, 'Queens', 45.11, 5.4),
  (2003, 1, 'Unknown', 3.3, 0.0),
  (2003, 3, 'Queens', 3.31, 0.4),
  (2003, 12, 'Manhattan', 7.3, 0.0),
  (2004, 4, 'Manhattan', 12.3, 21.43),
  (2007, 12, 'Manhattan', 22.75, 0.0),
  (2008, 8, 'Manhattan', 10.13, 15.43),
  (2008, 12, 'Bronx', 22.3, 0.0),
  (2008, 12, 'Brooklyn', 19.23, 3.94),
  (2008, 12, 'Manhattan', 17.47, 8.13),
  (2008, 12, 'Queens', 45.87, 9.19),
  (2008, 12, 'Unknown', 12.92, 5.13

In [ ]:
# D. TICKET PROMEDIO POR SERVICE_TYPE Y MES
print("ANÁLISIS D: Ticket Promedio por Service_Type y Mes")

# Ticket promedio por service_type y mes
ticket_monthly = (
    df.groupBy("year", "month", "service_type")
    .agg(F.round(F.avg("total_amount"), 2).alias("avg_ticket"))
    .orderBy("year", "month", "service_type")
)

print("TICKET PROMEDIO POR SERVICE_TYPE Y MES:")
ticket_monthly.show(50)

In [18]:
# D. TICKET PROMEDIO POR SERVICE_TYPE Y MES
print("ANÁLISIS D: Ticket Promedio por Service_Type y Mes")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    year,
    month,
    service_type,
    ROUND(AVG(total_amount), 2) AS avg_ticket
FROM analytics.obt_trips
WHERE service_type IS NOT NULL
    AND total_amount IS NOT NULL
GROUP BY year, month, service_type
ORDER BY year, month, service_type
"""
execute_snowflake_sql(query)

ANÁLISIS D: Ticket Promedio por Service_Type y Mes


([(2001, 1, 'yellow', 25.93),
  (2001, 2, 'yellow', 3.8),
  (2001, 8, 'yellow', 24.55),
  (2002, 2, 'yellow', 17.9),
  (2002, 10, 'yellow', 22.58),
  (2002, 12, 'yellow', 31.06),
  (2003, 1, 'yellow', 35.42),
  (2003, 3, 'yellow', 3.31),
  (2003, 12, 'yellow', 7.3),
  (2004, 4, 'yellow', 12.3),
  (2007, 12, 'yellow', 22.75),
  (2008, 8, 'yellow', 10.13),
  (2008, 12, 'green', 15.99),
  (2008, 12, 'yellow', 24.54),
  (2009, 1, 'green', 18.29),
  (2009, 1, 'yellow', 20.36),
  (2010, 8, 'yellow', 18.36),
  (2010, 9, 'green', 18.36),
  (2011, 1, 'yellow', 12.8),
  (2011, 2, 'yellow', 14.98),
  (2012, 9, 'green', 9.79),
  (2014, 11, 'yellow', 52.38),
  (2015, 1, 'green', 14.81),
  (2015, 1, 'yellow', 15.08),
  (2015, 2, 'green', 14.49),
  (2015, 2, 'yellow', 15.38),
  (2015, 3, 'green', 14.57),
  (2015, 3, 'yellow', 15.82),
  (2015, 4, 'green', 14.82),
  (2015, 4, 'yellow', 16.01),
  (2015, 5, 'green', 15.26),
  (2015, 5, 'yellow', 16.4),
  (2015, 6, 'green', 14.98),
  (2015, 6, 'yellow', 1

In [ ]:
# E. VIAJES POR HORA DEL DÍA Y DÍA DE SEMANA (PICOS)
print("ANÁLISIS E: Viajes por Hora del Día y Día de Semana (Picos)")

# Viajes por hora del día y día de semana para identificar picos
hourly_daily_patterns = (
    df.groupBy("pickup_hour", "day_of_week", "is_weekend")
    .agg(F.count("*").alias("total_trips"))
    .orderBy("pickup_hour", "day_of_week")
)

print("VIAJES POR HORA DEL DÍA Y DÍA DE SEMANA (PICOS):")
hourly_daily_patterns.show(50)

In [19]:
# E. VIAJES POR HORA DEL DÍA Y DÍA DE SEMANA (PICOS)
print("ANÁLISIS E: Viajes por Hora del Día y Día de Semana (Picos)")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pickup_hour,
    day_of_week,
    is_weekend,
    COUNT(*) AS total_trips
FROM analytics.obt_trips
WHERE pickup_hour IS NOT NULL
    AND day_of_week IS NOT NULL
    AND is_weekend IS NOT NULL
GROUP BY pickup_hour, day_of_week, is_weekend
ORDER BY pickup_hour, day_of_week;
"""
execute_snowflake_sql(query)

ANÁLISIS E: Viajes por Hora del Día y Día de Semana (Picos)


([(0, 0, False, 6482322),
  (0, 1, True, 2231648),
  (0, 2, False, 2250748),
  (0, 3, False, 2771714),
  (0, 4, False, 3274393),
  (0, 5, False, 4382436),
  (0, 6, False, 6339551),
  (1, 0, False, 5696055),
  (1, 1, True, 1326870),
  (1, 2, False, 1259670),
  (1, 3, False, 1565910),
  (1, 4, False, 1896310),
  (1, 5, False, 2780467),
  (1, 6, False, 5257342),
  (2, 0, False, 4441470),
  (2, 1, True, 849826),
  (2, 2, False, 757415),
  (2, 3, False, 948056),
  (2, 4, False, 1163793),
  (2, 5, False, 1770136),
  (2, 6, False, 4175562),
  (3, 0, False, 3417572),
  (3, 1, True, 611462),
  (3, 2, False, 510473),
  (3, 3, False, 633833),
  (3, 4, False, 777843),
  (3, 5, False, 1170632),
  (3, 6, False, 3020613),
  (4, 0, False, 2207100),
  (4, 1, True, 698586),
  (4, 2, False, 566501),
  (4, 3, False, 643814),
  (4, 4, False, 744362),
  (4, 5, False, 995251),
  (4, 6, False, 1942282),
  (5, 0, False, 1057875),
  (5, 1, True, 1128538),
  (5, 2, False, 1083916),
  (5, 3, False, 1108504),
  (5

In [ ]:
# F. P50/P90 DE TRIP_DURATION_MIN POR BOROUGH DE PICKUP
print("ANÁLISIS F: P50/P90 de Trip_Duration_Min por Borough de Pickup")

# P50/P90 de trip_duration_min por borough de pickup
duration_percentiles = (
    df.groupBy("pu_borough")
    .agg(F.round(F.expr("percentile_approx(trip_duration_min, 0.5)"), 2).alias("p50_duration"),
         F.round(F.expr("percentile_approx(trip_duration_min, 0.9)"), 2).alias("p90_duration"))
    .orderBy("pu_borough")
)

print("P50/P90 DE TRIP_DURATION_MIN POR BOROUGH DE PICKUP:")
duration_percentiles.show()

In [21]:
# F. P50/P90 DE TRIP_DURATION_MIN POR BOROUGH DE PICKUP
print("ANÁLISIS F: P50/P90 de Trip_Duration_Min por Borough de Pickup")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pu_borough,
    APPROX_PERCENTILE(trip_duration_min, 0.5) AS p50_duration,
    APPROX_PERCENTILE(trip_duration_min, 0.9) AS p90_duration
FROM analytics.obt_trips
WHERE pu_borough IS NOT NULL
  AND trip_duration_min > 0
GROUP BY pu_borough
ORDER BY pu_borough;
"""
execute_snowflake_sql(query)

ANÁLISIS F: P50/P90 de Trip_Duration_Min por Borough de Pickup


([('Bronx', 14.0, 38.997094005331824),
  ('Brooklyn', 13.000000000000004, 33.00000000000001),
  ('EWR', 1.0, 12.687074829931971),
  ('Manhattan', 11.000000000000007, 25.00000000000002),
  ('Queens', 25.000000000000014, 54.99870663581259),
  ('Staten Island', 28.0, 72.9800420168067),
  ('Unknown', 11.000000000000005, 29.000000000000007)],
 ['PU_BOROUGH', 'P50_DURATION', 'P90_DURATION'])

In [ ]:
# G. AVG_SPEED_MPH POR FRANJA HORARIA (6–9, 17–20) Y BOROUGH
print("ANÁLISIS G: Avg_Speed_Mph por Franja Horaria (6–9, 17–20) y Borough")

# Crear franja horaria y agrupar por franja y borough
speed_by_timeframe_borough = (
    df.withColumn("franja_horaria",
                  F.when((F.col("pickup_hour") >= 6) & (F.col("pickup_hour") <= 9), "Mañana (6-9h)")
                   .when((F.col("pickup_hour") >= 17) & (F.col("pickup_hour") <= 20), "Tarde (17-20h)")
                   .otherwise("Otras horas"))
    .filter(F.col("franja_horaria") != "Otras horas")
    .groupBy("franja_horaria", "pu_borough")
    .agg(F.round(F.avg("avg_speed_mph"), 2).alias("avg_speed_mph"))
    .orderBy("franja_horaria", "pu_borough")
)

print("AVG_SPEED_MPH POR FRANJA HORARIA (6–9, 17–20) Y BOROUGH:")
speed_by_timeframe_borough.show()

In [22]:
# G. AVG_SPEED_MPH POR FRANJA HORARIA (6–9, 17–20) Y BOROUGH
print("ANÁLISIS G: Avg_Speed_Mph por Franja Horaria (6–9, 17–20) y Borough")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    CASE
        WHEN pickup_hour BETWEEN 6 AND 9 THEN 'Mañana (6-9h)'
        WHEN pickup_hour BETWEEN 17 AND 20 THEN 'Tarde (17-20h)'
    END AS franja_horaria,
    pu_borough,
    ROUND(AVG(avg_speed_mph), 2) AS avg_speed_mph
FROM analytics.obt_trips
WHERE pickup_hour IS NOT NULL
    AND pu_borough IS NOT NULL
    AND avg_speed_mph IS NOT NULL
    AND (
        (pickup_hour BETWEEN 6 AND 9) OR
        (pickup_hour BETWEEN 17 AND 20)
    )
GROUP BY
    CASE
        WHEN pickup_hour BETWEEN 6 AND 9 THEN 'Mañana (6-9h)'
        WHEN pickup_hour BETWEEN 17 AND 20 THEN 'Tarde (17-20h)'
    END,
    pu_borough
ORDER BY franja_horaria, pu_borough;
"""
execute_snowflake_sql(query)

ANÁLISIS G: Avg_Speed_Mph por Franja Horaria (6–9, 17–20) y Borough


([('Mañana (6-9h)', 'Bronx', 144.11),
  ('Mañana (6-9h)', 'Brooklyn', 50.83),
  ('Mañana (6-9h)', 'EWR', 168.75),
  ('Mañana (6-9h)', 'Manhattan', 21.21),
  ('Mañana (6-9h)', 'Queens', 36.66),
  ('Mañana (6-9h)', 'Staten Island', 58.24),
  ('Mañana (6-9h)', 'Unknown', 31.97),
  ('Tarde (17-20h)', 'Bronx', 45.33),
  ('Tarde (17-20h)', 'Brooklyn', 21.34),
  ('Tarde (17-20h)', 'EWR', 189.87),
  ('Tarde (17-20h)', 'Manhattan', 39.1),
  ('Tarde (17-20h)', 'Queens', 34.07),
  ('Tarde (17-20h)', 'Staten Island', 95.02),
  ('Tarde (17-20h)', 'Unknown', 12.13)],
 ['FRANJA_HORARIA', 'PU_BOROUGH', 'AVG_SPEED_MPH'])

In [ ]:
# H. PARTICIPACIÓN POR PAYMENT_TYPE_DESC Y SU RELACIÓN CON TIP_PCT
print("ANÁLISIS H: Participación por Payment_Type_Desc y su Relación con Tip_Pct")

# Participación por payment_type_desc y su relación con tip_pct
payment_participation_tips = (
    df.groupBy("payment_type_desc")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("tip_pct"), 2).alias("avg_tip_pct"),
         F.round(F.count("*") * 100.0 / df.count(), 2).alias("participacion_pct"))
    .orderBy(F.col("total_trips").desc())
)

print("PARTICIPACIÓN POR PAYMENT_TYPE_DESC Y SU RELACIÓN CON TIP_PCT:")
payment_participation_tips.show()

In [23]:
# H. PARTICIPACIÓN POR PAYMENT_TYPE_DESC Y SU RELACIÓN CON TIP_PCT
print("ANÁLISIS H: Participación por Payment_Type_Desc y su Relación con Tip_Pct")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH total_dataset AS (
    SELECT COUNT(*) as total_records
    FROM analytics.obt_trips
    WHERE payment_type_desc IS NOT NULL
)
SELECT
    payment_type_desc,
    COUNT(*) AS total_trips,
    ROUND(AVG(tip_pct), 2) AS avg_tip_pct,
    ROUND((COUNT(*) * 100.0 / (SELECT total_records FROM total_dataset)), 2) AS participacion_pct
FROM analytics.obt_trips
WHERE payment_type_desc IS NOT NULL
    AND tip_pct IS NOT NULL
GROUP BY payment_type_desc
ORDER BY COUNT(*) DESC;
"""
execute_snowflake_sql(query)

ANÁLISIS H: Participación por Payment_Type_Desc y su Relación con Tip_Pct


([('Credit card', 567185276, 24.48, Decimal('67.31')),
  ('Cash', 252051943, 0.0, Decimal('29.91')),
  ('Flex Fare trip', 16674850, 5.62, Decimal('1.98')),
  ('No charge', 2967048, 0.05, Decimal('0.35')),
  ('Dispute', 1954872, 0.07, Decimal('0.23')),
  ('Unknown', 1808844, 4.19, Decimal('0.21'))],
 ['PAYMENT_TYPE_DESC', 'TOTAL_TRIPS', 'AVG_TIP_PCT', 'PARTICIPACION_PCT'])

In [ ]:
# I. ¿QUÉ RATE_CODE_DESC CONCENTRAN MAYOR TRIP_DISTANCE Y TOTAL_AMOUNT?
print("ANÁLISIS I: ¿Qué Rate_Code_Desc concentran mayor Trip_Distance y Total_Amount?")

# Análisis de rate_code_desc por trip_distance y total_amount
rate_code_analysis = (
    df.groupBy("rate_code_desc")
    .agg(F.round(F.avg("trip_distance"), 2).alias("avg_trip_distance"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.count("*").alias("total_trips"))
    .orderBy(F.col("avg_trip_distance").desc(), F.col("avg_total_amount").desc())
)

print("¿QUÉ RATE_CODE_DESC CONCENTRAN MAYOR TRIP_DISTANCE Y TOTAL_AMOUNT?:")
rate_code_analysis.show()

In [24]:
# I. ¿QUÉ RATE_CODE_DESC CONCENTRAN MAYOR TRIP_DISTANCE Y TOTAL_AMOUNT?
print("ANÁLISIS I: ¿Qué Rate_Code_Desc concentran mayor Trip_Distance y Total_Amount?")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    rate_code_desc,
    ROUND(AVG(trip_distance), 2) AS avg_trip_distance,
    ROUND(AVG(total_amount), 2) AS avg_total_amount,
    COUNT(*) AS total_trips
FROM analytics.obt_trips
WHERE rate_code_desc IS NOT NULL
    AND trip_distance IS NOT NULL
    AND total_amount IS NOT NULL
GROUP BY rate_code_desc
ORDER BY AVG(trip_distance) DESC, AVG(total_amount) DESC;
"""
execute_snowflake_sql(query)

ANÁLISIS I: ¿Qué Rate_Code_Desc concentran mayor Trip_Distance y Total_Amount?


([('Unknown', 41.2, 26.58, 18481466),
  ('Nassau or Westchester', 29.35, 101.14, 688153),
  ('Negotiated fare', 26.98, 60.38, 3324900),
  ('Null/unknown', 24.02, 37.68, 1216003),
  ('JFK', 22.64, 71.64, 18827410),
  ('Newark', 17.39, 98.06, 1616402),
  ('Standard rate', 4.4, 16.62, 798485788),
  ('Group ride', 1.82, 28.66, 2711)],
 ['RATE_CODE_DESC', 'AVG_TRIP_DISTANCE', 'AVG_TOTAL_AMOUNT', 'TOTAL_TRIPS'])

In [ ]:
# J. MIX YELLOW VS GREEN POR MES Y BOROUGH
print("ANÁLISIS J: Mix Yellow vs Green por Mes y Borough")

# Análisis del mix de servicios yellow vs green por mes y borough de pickup
yellow_green_mix_monthly = (
    df.filter(F.col("service_type").isin(["yellow", "green"]))
    .groupBy("year", "month", "pu_borough", "service_type")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_fare"))
    .withColumn("total_by_month_borough", F.sum("total_trips").over(F.Window.partitionBy("year", "month", "pu_borough")))
    .withColumn("service_mix_pct", F.round((F.col("total_trips") * 100.0 / F.col("total_by_month_borough")), 2))
    .orderBy("year", "month", "pu_borough", "service_type")
)

print("MIX YELLOW VS GREEN POR MES Y BOROUGH:")
yellow_green_mix_monthly.show(50)

In [25]:
# J. MIX YELLOW VS GREEN POR MES Y BOROUGH
print("ANÁLISIS J: Mix Yellow vs Green por Mes y Borough")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH service_counts AS (
    SELECT
        year,
        month,
        pu_borough,
        service_type,
        COUNT(*) as total_trips,
        ROUND(AVG(total_amount), 2) as avg_fare
    FROM analytics.obt_trips
    WHERE service_type IN ('yellow', 'green')
        AND pu_borough IS NOT NULL
    GROUP BY year, month, pu_borough, service_type
),
monthly_totals AS (
    SELECT
        year,
        month,
        pu_borough,
        SUM(total_trips) as total_by_month_borough
    FROM service_counts
    GROUP BY year, month, pu_borough
)
SELECT
    sc.year,
    sc.month,
    sc.pu_borough,
    sc.service_type,
    sc.total_trips,
    sc.avg_fare,
    mt.total_by_month_borough,
    ROUND((sc.total_trips * 100.0 / mt.total_by_month_borough), 2) as service_mix_pct
FROM service_counts sc
JOIN monthly_totals mt
    ON sc.year = mt.year
    AND sc.month = mt.month
    AND sc.pu_borough = mt.pu_borough
ORDER BY sc.year, sc.month, sc.pu_borough, sc.service_type;
"""
execute_snowflake_sql(query)

ANÁLISIS J: Mix Yellow vs Green por Mes y Borough


([(2001, 1, 'Brooklyn', 'yellow', 1, 8.8, 1, Decimal('100.00')),
  (2001, 1, 'Manhattan', 'yellow', 17, 19.88, 17, Decimal('100.00')),
  (2001, 1, 'Queens', 'yellow', 5, 49.9, 5, Decimal('100.00')),
  (2001, 2, 'Queens', 'yellow', 1, 3.8, 1, Decimal('100.00')),
  (2001, 8, 'Queens', 'yellow', 1, 24.55, 1, Decimal('100.00')),
  (2002, 2, 'Manhattan', 'yellow', 11, 17.9, 11, Decimal('100.00')),
  (2002, 10, 'Brooklyn', 'yellow', 6, 28.59, 6, Decimal('100.00')),
  (2002, 10, 'Manhattan', 'yellow', 196, 18.33, 196, Decimal('100.00')),
  (2002, 10, 'Queens', 'yellow', 22, 51.39, 22, Decimal('100.00')),
  (2002, 10, 'Unknown', 'yellow', 8, 21.15, 8, Decimal('100.00')),
  (2002, 12, 'Brooklyn', 'yellow', 1, 31.3, 1, Decimal('100.00')),
  (2002, 12, 'Manhattan', 'yellow', 22, 19.39, 22, Decimal('100.00')),
  (2002, 12, 'Queens', 'yellow', 18, 48.09, 18, Decimal('100.00')),
  (2002, 12, 'Unknown', 'yellow', 2, 6.03, 2, Decimal('100.00')),
  (2003, 1, 'Manhattan', 'yellow', 18, 28.6, 18, Decimal

In [ ]:
# K. TOP 20 FLUJOS PU→DO POR VOLUMEN Y SU TICKET PROMEDIO
print("ANÁLISIS K: Top 20 Flujos PU→DO por Volumen y su Ticket Promedio")

# Top 20 flujos pickup→dropoff por volumen y ticket promedio
top_flows_volume_ticket = (
    df.groupBy("pu_borough", "do_borough")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_ticket"))
    .orderBy(F.col("total_trips").desc())
)

print("TOP 20 FLUJOS PU→DO POR VOLUMEN Y SU TICKET PROMEDIO:")
top_flows_volume_ticket.show(20)

In [26]:
# K. TOP 20 FLUJOS PU→DO POR VOLUMEN Y SU TICKET PROMEDIO
print("ANÁLISIS K: Top 20 Flujos PU→DO por Volumen y su Ticket Promedio")


# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pu_borough,
    do_borough,
    COUNT(*) AS total_trips,
    ROUND(AVG(total_amount), 2) AS avg_ticket
FROM analytics.obt_trips
WHERE pu_borough IS NOT NULL
    AND do_borough IS NOT NULL
    AND total_amount IS NOT NULL
GROUP BY pu_borough, do_borough
ORDER BY COUNT(*) DESC
LIMIT 20;
"""
execute_snowflake_sql(query)

ANÁLISIS K: Top 20 Flujos PU→DO por Volumen y su Ticket Promedio


([('Manhattan', 'Manhattan', 668253309, 14.32),
  ('Queens', 'Manhattan', 30467838, 57.48),
  ('Queens', 'Queens', 29138764, 18.59),
  ('Manhattan', 'Queens', 25568621, 45.65),
  ('Brooklyn', 'Brooklyn', 25043478, 14.02),
  ('Manhattan', 'Brooklyn', 21535025, 30.4),
  ('Queens', 'Brooklyn', 8144013, 48.76),
  ('Unknown', 'Unknown', 7297522, 16.76),
  ('Brooklyn', 'Manhattan', 7056642, 27.18),
  ('Manhattan', 'Bronx', 5294209, 28.49),
  ('Bronx', 'Bronx', 3236524, 14.64),
  ('Brooklyn', 'Queens', 2183569, 33.36),
  ('Unknown', 'Manhattan', 1485897, 19.89),
  ('Manhattan', 'EWR', 1416351, 101.52),
  ('Bronx', 'Manhattan', 1321043, 26.05),
  ('Queens', 'Bronx', 1081359, 58.0),
  ('Manhattan', 'Unknown', 506313, 21.88),
  ('Brooklyn', 'Bronx', 189460, 61.26),
  ('Bronx', 'Queens', 187677, 49.43),
  ('Bronx', 'Brooklyn', 150062, 63.89)],
 ['PU_BOROUGH', 'DO_BOROUGH', 'TOTAL_TRIPS', 'AVG_TICKET'])

In [ ]:
# L. DISTRIBUCIÓN DE PASSENGER_COUNT Y EFECTO EN TOTAL_AMOUNT
print("ANÁLISIS L: Distribución de Passenger_Count y Efecto en Total_Amount")

# Distribución de passenger_count y su efecto en total_amount
passenger_distribution_effect = (
    df.groupBy("passenger_count")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.round(F.count("*") * 100.0 / df.count(), 2).alias("participacion_pct"))
    .orderBy("passenger_count")
)

print("DISTRIBUCIÓN DE PASSENGER_COUNT Y EFECTO EN TOTAL_AMOUNT:")
passenger_distribution_effect.show()

In [27]:
# L. DISTRIBUCIÓN DE PASSENGER_COUNT Y EFECTO EN TOTAL_AMOUNT
print("ANÁLISIS L: Distribución de Passenger_Count y Efecto en Total_Amount")


# Consulta SQL directa usando conector de Snowflake
query = """
WITH total_dataset AS (
    SELECT COUNT(*) as total_records
    FROM analytics.obt_trips
    WHERE passenger_count IS NOT NULL
)
SELECT
    passenger_count,
    COUNT(*) AS total_trips,
    ROUND(AVG(total_amount), 2) AS avg_total_amount,
    ROUND((COUNT(*) * 100.0 / (SELECT total_records FROM total_dataset)), 2) AS participacion_pct
FROM analytics.obt_trips
WHERE passenger_count IS NOT NULL
    AND total_amount IS NOT NULL
GROUP BY passenger_count
ORDER BY passenger_count;
"""
execute_snowflake_sql(query)

ANÁLISIS L: Distribución de Passenger_Count y Efecto en Total_Amount


([(0, 5733919, 19.49, Decimal('0.70')),
  (1, 600557382, 18.13, Decimal('72.87')),
  (2, 116250233, 19.49, Decimal('14.11')),
  (3, 32262614, 18.93, Decimal('3.91')),
  (4, 15532695, 19.77, Decimal('1.88')),
  (5, 33326377, 17.04, Decimal('4.04')),
  (6, 20492805, 16.88, Decimal('2.49')),
  (7, 2091, 49.8, Decimal('0.00')),
  (8, 2097, 54.36, Decimal('0.00')),
  (9, 1147, 69.74, Decimal('0.00')),
  (32, 1, 60.35, Decimal('0.00')),
  (48, 1, 40.3, Decimal('0.00')),
  (96, 2, 12.59, Decimal('0.00')),
  (112, 1, 14.76, Decimal('0.00')),
  (192, 2, 9.15, Decimal('0.00'))],
 ['PASSENGER_COUNT', 'TOTAL_TRIPS', 'AVG_TOTAL_AMOUNT', 'PARTICIPACION_PCT'])

In [ ]:
# M. IMPACTO DE TOLLS_AMOUNT Y CONGESTION_SURCHARGE POR ZONA
print("ANÁLISIS M: Impacto de Tolls_Amount y Congestion_Surcharge por Zona")

# Impacto de tolls_amount y congestion_surcharge por zona de pickup
tolls_congestion_impact_by_zone = (
    df.groupBy("pu_zone", "pu_borough")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("tolls_amount"), 2).alias("avg_tolls_amount"),
         F.round(F.avg("congestion_surcharge"), 2).alias("avg_congestion_surcharge"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"))
    .orderBy(F.col("avg_tolls_amount").desc(), F.col("avg_congestion_surcharge").desc())
)

print("IMPACTO DE TOLLS_AMOUNT Y CONGESTION_SURCHARGE POR ZONA:")
tolls_congestion_impact_by_zone.show(20)

In [28]:
# M. IMPACTO DE TOLLS_AMOUNT Y CONGESTION_SURCHARGE POR ZONA
print("ANÁLISIS M: Impacto de Tolls_Amount y Congestion_Surcharge por Zona")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pu_zone,
    pu_borough,
    COUNT(*) AS total_trips,
    ROUND(AVG(tolls_amount), 2) AS avg_tolls_amount,
    ROUND(AVG(congestion_surcharge), 2) AS avg_congestion_surcharge,
    ROUND(AVG(total_amount), 2) AS avg_total_amount
FROM analytics.obt_trips
WHERE pu_zone IS NOT NULL
    AND pu_borough IS NOT NULL
GROUP BY pu_zone, pu_borough
ORDER BY AVG(tolls_amount) DESC, AVG(congestion_surcharge) DESC
LIMIT 20;
"""
execute_snowflake_sql(query)

ANÁLISIS M: Impacto de Tolls_Amount y Congestion_Surcharge por Zona


([('Rikers Island', 'Bronx', 324, 26.34, 1.21, 53.92),
  ('Arden Heights', 'Staten Island', 2118, 14.41, 0.12, 85.74),
  ('Arrochar/Fort Wadsworth', 'Staten Island', 3963, 9.02, 0.11, 34.63),
  ('Bloomfield/Emerson Hill', 'Staten Island', 6937, 7.78, 0.04, 70.61),
  ('Charleston/Tottenville', 'Staten Island', 3749, 7.24, 0.01, 91.43),
  ("Eltingville/Annadale/Prince's Bay",
   'Staten Island',
   440,
   6.63,
   0.11,
   71.6),
  ('Rossville/Woodrow', 'Staten Island', 304, 6.2, 0.09, 83.95),
  ('South Beach/Dongan Hills', 'Staten Island', 1842, 5.78, 0.09, 49.21),
  ('Great Kills', 'Staten Island', 510, 5.31, 0.17, 59.54),
  ('Oakwood', 'Staten Island', 395, 5.19, 0.29, 55.26),
  ('Grymes Hill/Clifton', 'Staten Island', 5023, 5.11, 0.07, 44.82),
  ('Mariners Harbor', 'Staten Island', 4650, 4.96, 0.07, 49.61),
  ('New Dorp/Midland Beach', 'Staten Island', 922, 4.84, 0.21, 46.02),
  ('Westerleigh', 'Staten Island', 1435, 4.69, 0.26, 47.27),
  ('Newark Airport', 'EWR', 30065, 4.6, 0.06, 

In [ ]:
# N. PROPORCIÓN DE VIAJES CORTOS VS LARGOS POR BOROUGH Y ESTACIONALIDAD
print("ANÁLISIS N: Proporción de Viajes Cortos vs Largos por Borough y Estacionalidad")

# Definir categorías de viaje (corto ≤ 3 millas, largo > 3 millas) y analizar por borough y estación
trip_length_seasonality = (
    df.withColumn("trip_category",
                  F.when(F.col("trip_distance") <= 3, "Corto (≤3 mi)")
                   .otherwise("Largo (>3 mi)"))
    .withColumn("season",
                F.when(F.col("month").isin([12, 1, 2]), "Invierno")
                 .when(F.col("month").isin([3, 4, 5]), "Primavera")
                 .when(F.col("month").isin([6, 7, 8]), "Verano")
                 .otherwise("Otoño"))
    .groupBy("pu_borough", "season", "trip_category")
    .agg(F.count("*").alias("total_trips"))
    .withColumn("total_by_borough_season", F.sum("total_trips").over(F.Window.partitionBy("pu_borough", "season")))
    .withColumn("proportion_pct", F.round((F.col("total_trips") * 100.0 / F.col("total_by_borough_season")), 2))
    .orderBy("pu_borough", "season", "trip_category")
)

print("PROPORCIÓN DE VIAJES CORTOS VS LARGOS POR BOROUGH Y ESTACIONALIDAD:")
trip_length_seasonality.show(50)

In [30]:
# N. PROPORCIÓN DE VIAJES CORTOS VS LARGOS POR BOROUGH Y ESTACIONALIDAD
print("ANÁLISIS N: Proporción de Viajes Cortos vs Largos por Borough y Estacionalidad")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH trips_enriched AS (
    SELECT
        pu_borough,
        CASE
            WHEN month IN (12, 1, 2) THEN 'Invierno'
            WHEN month IN (3, 4, 5) THEN 'Primavera'
            WHEN month IN (6, 7, 8) THEN 'Verano'
            ELSE 'Otoño'
        END AS season,
        CASE
            WHEN trip_distance <= 3 THEN 'Corto (≤3 mi)'
            ELSE 'Largo (>3 mi)'
        END AS trip_category
    FROM analytics.obt_trips
    WHERE pu_borough IS NOT NULL
      AND trip_distance IS NOT NULL
)

SELECT
    pu_borough,
    season,
    trip_category,
    COUNT(*) AS total_trips,
    ROUND(
        COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY pu_borough, season),
        2
    ) AS proportion_pct
FROM trips_enriched
GROUP BY pu_borough, season, trip_category
ORDER BY pu_borough, season, trip_category;

"""
execute_snowflake_sql(query)

ANÁLISIS N: Proporción de Viajes Cortos vs Largos por Borough y Estacionalidad


([('Bronx', 'Invierno', 'Corto (≤3 mi)', 689069, Decimal('54.53')),
  ('Bronx', 'Invierno', 'Largo (>3 mi)', 574594, Decimal('45.47')),
  ('Bronx', 'Otoño', 'Corto (≤3 mi)', 550997, Decimal('52.32')),
  ('Bronx', 'Otoño', 'Largo (>3 mi)', 502062, Decimal('47.68')),
  ('Bronx', 'Primavera', 'Corto (≤3 mi)', 795797, Decimal('55.32')),
  ('Bronx', 'Primavera', 'Largo (>3 mi)', 642646, Decimal('44.68')),
  ('Bronx', 'Verano', 'Corto (≤3 mi)', 630985, Decimal('52.86')),
  ('Bronx', 'Verano', 'Largo (>3 mi)', 562814, Decimal('47.14')),
  ('Brooklyn', 'Invierno', 'Corto (≤3 mi)', 5283719, Decimal('60.19')),
  ('Brooklyn', 'Invierno', 'Largo (>3 mi)', 3494415, Decimal('39.81')),
  ('Brooklyn', 'Otoño', 'Corto (≤3 mi)', 4498646, Decimal('58.36')),
  ('Brooklyn', 'Otoño', 'Largo (>3 mi)', 3210313, Decimal('41.64')),
  ('Brooklyn', 'Primavera', 'Corto (≤3 mi)', 5683658, Decimal('58.29')),
  ('Brooklyn', 'Primavera', 'Largo (>3 mi)', 4066178, Decimal('41.71')),
  ('Brooklyn', 'Verano', 'Corto (≤3 

In [ ]:
# O. DIFERENCIAS POR VENDOR EN AVG_SPEED_MPH Y TRIP_DURATION_MIN
print("ANÁLISIS O: Diferencias por Vendor en Avg_Speed_Mph y Trip_Duration_Min")

# Análisis de diferencias por vendor en velocidad promedio y duración de viaje
vendor_performance_comparison = (
    df.groupBy("vendor_name")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("avg_speed_mph"), 2).alias("avg_speed_mph"),
         F.round(F.avg("trip_duration_min"), 2).alias("avg_duration_min"),
         F.round(F.avg("trip_distance"), 2).alias("avg_distance"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"))
    .orderBy(F.col("avg_speed_mph").desc())
)

print("DIFERENCIAS POR VENDOR EN AVG_SPEED_MPH Y TRIP_DURATION_MIN:")
vendor_performance_comparison.show()

In [32]:
# O. DIFERENCIAS POR VENDOR EN AVG_SPEED_MPH Y TRIP_DURATION_MIN
print("ANÁLISIS O: Diferencias por Vendor en Avg_Speed_Mph y Trip_Duration_Min")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    vendor_name,
    COUNT(*) AS total_trips,
    ROUND(AVG(avg_speed_mph), 2) AS avg_speed_mph,
    ROUND(AVG(trip_duration_min), 2) AS avg_duration_min,
    ROUND(AVG(trip_distance), 2) AS avg_distance,
    ROUND(AVG(total_amount), 2) AS avg_total_amount
FROM analytics.obt_trips
WHERE vendor_name IS NOT NULL
GROUP BY vendor_name
ORDER BY avg_speed_mph DESC;
"""
execute_snowflake_sql(query)

ANÁLISIS O: Diferencias por Vendor en Avg_Speed_Mph y Trip_Duration_Min


([('Creative Mobile Technologies, LLC', 319950699, 42.08, 14.41, 7.88, 17.46),
  ('Curb Mobility, LLC', 521791437, 17.18, 18.7, 4.49, 19.14),
  ('Unknown', 763798, 10.69, 14.3, 2.75, 16.62),
  ('Myle Technologies Inc', 136899, 10.39, 61.73, 10.23, 46.08)],
 ['VENDOR_NAME',
  'TOTAL_TRIPS',
  'AVG_SPEED_MPH',
  'AVG_DURATION_MIN',
  'AVG_DISTANCE',
  'AVG_TOTAL_AMOUNT'])

In [ ]:
# P. RELACIÓN MÉTODO DE PAGO ↔ TIP_AMOUNT POR HORA
print("ANÁLISIS P: Relación Método de Pago ↔ Tip_Amount por Hora")

# Análisis de la relación entre método de pago y cantidad de propina por hora del día
payment_tip_hourly_relationship = (
    df.groupBy("pickup_hour", "payment_type_desc")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("tip_amount"), 2).alias("avg_tip_amount"),
         F.round(F.avg("tip_pct"), 2).alias("avg_tip_pct"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"))
    .orderBy("pickup_hour", F.col("avg_tip_amount").desc())
)

print("RELACIÓN MÉTODO DE PAGO ↔ TIP_AMOUNT POR HORA:")
payment_tip_hourly_relationship.show(50)

In [33]:
# P. RELACIÓN MÉTODO DE PAGO ↔ TIP_AMOUNT POR HORA
print("ANÁLISIS P: Relación Método de Pago ↔ Tip_Amount por Hora")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pickup_hour,
    payment_type_desc,
    COUNT(*) AS total_trips,
    ROUND(AVG(tip_amount), 2) AS avg_tip_amount,
    ROUND(AVG(tip_pct), 2) AS avg_tip_pct,
    ROUND(AVG(total_amount), 2) AS avg_total_amount
FROM analytics.obt_trips
WHERE pickup_hour IS NOT NULL
    AND payment_type_desc IS NOT NULL
GROUP BY pickup_hour, payment_type_desc
ORDER BY pickup_hour, AVG(tip_amount) DESC
LIMIT 50;
"""
execute_snowflake_sql(query)

ANÁLISIS P: Relación Método de Pago ↔ Tip_Amount por Hora


([(0, 'Credit card', 18707363, 3.0, 25.53, 20.04),
  (0, 'Unknown', 11922, 1.69, 8.14, 31.85),
  (0, 'Flex Fare trip', 702500, 0.74, 3.82, 23.76),
  (0, 'Dispute', 74846, 0.01, 0.07, 22.91),
  (0, 'No charge', 118663, 0.0, 0.02, 16.88),
  (0, 'Cash', 8117518, 0.0, 0.0, 14.73),
  (1, 'Credit card', 13197112, 2.78, 26.57, 18.69),
  (1, 'Unknown', 7393, 1.59, 7.73, 36.91),
  (1, 'Flex Fare trip', 463582, 0.71, 3.66, 23.73),
  (1, 'Dispute', 56420, 0.01, 0.05, 21.48),
  (1, 'No charge', 97244, 0.0, 0.04, 23.87),
  (1, 'Cash', 5960873, 0.0, 0.0, 14.44),
  (2, 'Credit card', 9262265, 2.64, 28.22, 17.89),
  (2, 'Unknown', 5442, 1.62, 7.54, 37.98),
  (2, 'Flex Fare trip', 317820, 0.75, 3.84, 24.01),
  (2, 'Dispute', 43416, 0.01, 0.06, 20.61),
  (2, 'No charge', 81180, 0.0, 0.05, 22.46),
  (2, 'Cash', 4396135, 0.0, 0.0, 14.32),
  (3, 'Credit card', 6355523, 2.69, 27.37, 18.51),
  (3, 'Unknown', 8295, 1.21, 4.76, 41.29),
  (3, 'Flex Fare trip', 243841, 0.94, 4.09, 26.2),
  (3, 'Dispute', 35470, 

In [ ]:
# Q. ZONAS CON PERCENTIL 99 DE DURACIÓN/DISTANCIA FUERA DE RANGO (POSIBLE CONGESTIÓN/EVENTOS)
print("ANÁLISIS Q: Zonas con Percentil 99 de Duración/Distancia fuera de Rango (Posible Congestión/Eventos)")

# Identificar zonas con P99 extremos que podrían indicar congestión severa o eventos especiales
zones_p99_outliers = (
    df.groupBy("pu_zone", "pu_borough")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.expr("percentile_approx(trip_duration_min, 0.99)"), 2).alias("p99_duration"),
         F.round(F.expr("percentile_approx(trip_distance, 0.99)"), 2).alias("p99_distance"),
         F.round(F.avg("trip_duration_min"), 2).alias("avg_duration"),
         F.round(F.avg("trip_distance"), 2).alias("avg_distance"),
         F.round(F.avg("avg_speed_mph"), 2).alias("avg_speed"))
    .withColumn("duration_outlier_ratio", F.round(F.col("p99_duration") / F.col("avg_duration"), 2))
    .withColumn("distance_outlier_ratio", F.round(F.col("p99_distance") / F.col("avg_distance"), 2))
    .filter((F.col("duration_outlier_ratio") > 5) | (F.col("distance_outlier_ratio") > 4) | (F.col("p99_duration") > 120))
    .orderBy(F.col("duration_outlier_ratio").desc(), F.col("distance_outlier_ratio").desc())
)

print("ZONAS CON PERCENTIL 99 DE DURACIÓN/DISTANCIA FUERA DE RANGO:")
zones_p99_outliers.show(20)

In [35]:
# Q. ZONAS CON PERCENTIL 99 DE DURACIÓN/DISTANCIA FUERA DE RANGO (POSIBLE CONGESTIÓN/EVENTOS)
print("ANÁLISIS Q: Zonas con Percentil 99 de Duración/Distancia fuera de Rango (Posible Congestión/Eventos)")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH zone_metrics AS (
    SELECT
        pu_zone,
        pu_borough,
        COUNT(*) AS total_trips,
        APPROX_PERCENTILE(trip_duration_min, 0.99) AS p99_duration,
        APPROX_PERCENTILE(trip_distance, 0.99) AS p99_distance,
        ROUND(AVG(trip_duration_min), 2) AS avg_duration,
        ROUND(AVG(trip_distance), 2) AS avg_distance,
        ROUND(AVG(avg_speed_mph), 2) AS avg_speed
    FROM analytics.obt_trips
    WHERE pu_zone IS NOT NULL
      AND pu_borough IS NOT NULL
      AND trip_duration_min > 0
      AND trip_distance > 0
    GROUP BY pu_zone, pu_borough
)
SELECT
    pu_zone,
    pu_borough,
    total_trips,
    ROUND(p99_duration, 2) AS p99_duration,
    ROUND(p99_distance, 2) AS p99_distance,
    avg_duration,
    avg_distance,
    avg_speed,
    ROUND(p99_duration / NULLIF(avg_duration, 0), 2) AS duration_outlier_ratio,
    ROUND(p99_distance / NULLIF(avg_distance, 0), 2) AS distance_outlier_ratio
FROM zone_metrics
WHERE (p99_duration / NULLIF(avg_duration, 0) > 5)
   OR (p99_distance / NULLIF(avg_distance, 0) > 4)
   OR (p99_duration > 120)
ORDER BY duration_outlier_ratio DESC, distance_outlier_ratio DESC
LIMIT 20;
"""
execute_snowflake_sql(query)

ANÁLISIS Q: Zonas con Percentil 99 de Duración/Distancia fuera de Rango (Posible Congestión/Eventos)


([('Bronx Park',
   'Bronx',
   31358,
   246.23,
   22.19,
   32.37,
   14.62,
   38.16,
   7.61,
   1.52),
  ('Newark Airport',
   'EWR',
   11893,
   97.48,
   37.26,
   15.26,
   7.28,
   181.79,
   6.39,
   5.12),
  ('Rossville/Woodrow',
   'Staten Island',
   250,
   327.87,
   47.08,
   53.7,
   22.84,
   58.59,
   6.11,
   2.06),
  ('West Brighton',
   'Staten Island',
   1375,
   155.94,
   42.03,
   26.97,
   7.85,
   19.41,
   5.78,
   5.35),
  ('Coney Island',
   'Brooklyn',
   175552,
   180.02,
   30.15,
   39.88,
   29.17,
   73.75,
   4.51,
   1.03),
  ('Broad Channel',
   'Queens',
   686,
   126.9,
   31.51,
   28.84,
   9.63,
   22.08,
   4.4,
   3.27),
  ('Saint George/New Brighton',
   'Staten Island',
   4756,
   133.94,
   35.19,
   31.27,
   70.21,
   1509.52,
   4.28,
   0.5),
  ('Marine Park/Floyd Bennett Field',
   'Brooklyn',
   5874,
   132.0,
   50.2,
   30.92,
   9.85,
   23.58,
   4.27,
   5.1),
  ('Brighton Beach',
   'Brooklyn',
   57716,
   122.39,
  

In [ ]:
# R. YIELD POR MILLA (TOTAL_AMOUNT/TRIP_DISTANCE) POR BOROUGH Y HORA
print("ANÁLISIS R: Yield por Milla (Total_Amount/Trip_Distance) por Borough y Hora")

# Análisis del yield por milla (rentabilidad por distancia) por borough y hora del día
yield_per_mile_by_borough_hour = (
    df.filter(F.col("trip_distance") > 0)  # Evitar divisiones por cero
    .groupBy("pu_borough", "pickup_hour")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg(F.col("total_amount") / F.col("trip_distance")), 2).alias("yield_per_mile"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.round(F.avg("trip_distance"), 2).alias("avg_trip_distance"),
         F.round(F.avg("tip_pct"), 2).alias("avg_tip_pct"))
    .orderBy("pu_borough", F.col("yield_per_mile").desc())
)

print("YIELD POR MILLA (TOTAL_AMOUNT/TRIP_DISTANCE) POR BOROUGH Y HORA:")
yield_per_mile_by_borough_hour.show(50)

In [36]:
# R. YIELD POR MILLA (TOTAL_AMOUNT/TRIP_DISTANCE) POR BOROUGH Y HORA
print("ANÁLISIS R: Yield por Milla (Total_Amount/Trip_Distance) por Borough y Hora")

# Consulta SQL directa usando conector de Snowflake
query = """
SELECT
    pu_borough,
    pickup_hour,
    COUNT(*) AS total_trips,
    ROUND(AVG(total_amount / trip_distance), 2) AS yield_per_mile,
    ROUND(AVG(total_amount), 2) AS avg_total_amount,
    ROUND(AVG(trip_distance), 2) AS avg_trip_distance,
    ROUND(AVG(tip_pct), 2) AS avg_tip_pct
FROM analytics.obt_trips
WHERE trip_distance > 0
    AND pu_borough IS NOT NULL
    AND pickup_hour IS NOT NULL
GROUP BY pu_borough, pickup_hour
ORDER BY pu_borough, AVG(total_amount / trip_distance) DESC
LIMIT 50;
"""
execute_snowflake_sql(query)

ANÁLISIS R: Yield por Milla (Total_Amount/Trip_Distance) por Borough y Hora


([('Bronx', 17, 279380, 14.46, 19.47, 12.53, 3.48),
  ('Bronx', 16, 273155, 13.57, 20.72, 15.39, 11.39),
  ('Bronx', 18, 254147, 13.27, 18.3, 14.1, 3.67),
  ('Bronx', 19, 215654, 11.75, 18.12, 13.54, 4.24),
  ('Bronx', 3, 58314, 10.91, 19.51, 12.31, 4.01),
  ('Bronx', 1, 95182, 10.7, 16.18, 5.93, 4.65),
  ('Bronx', 2, 67760, 10.1, 16.7, 14.86, 4.17),
  ('Bronx', 20, 191334, 9.4, 18.42, 12.78, 3.94),
  ('Bronx', 15, 279090, 9.31, 20.36, 20.36, 4.18),
  ('Bronx', 0, 129138, 9.29, 16.61, 5.55, 6.75),
  ('Bronx', 22, 174568, 8.99, 19.3, 15.4, 4.69),
  ('Bronx', 14, 271785, 8.97, 20.67, 24.39, 2.91),
  ('Bronx', 23, 161506, 8.92, 18.38, 10.78, 4.33),
  ('Bronx', 4, 72353, 8.88, 23.61, 71.7, 3.2),
  ('Bronx', 5, 87979, 8.82, 28.53, 68.84, 11.14),
  ('Bronx', 21, 182857, 8.78, 19.04, 15.52, 4.43),
  ('Bronx', 13, 251270, 8.76, 21.03, 25.56, 2.98),
  ('Bronx', 10, 256123, 8.71, 21.49, 35.72, 3.0),
  ('Bronx', 11, 245566, 8.67, 21.55, 28.39, 2.94),
  ('Bronx', 9, 284597, 8.53, 21.25, 31.05, 3.8

In [ ]:
# S. CAMBIOS YOY EN VOLUMEN Y TICKET PROMEDIO POR SERVICE_TYPE
print("ANÁLISIS S: Cambios YoY en Volumen y Ticket Promedio por Service_Type")

# Análisis de cambios año a año en volumen de viajes y ticket promedio por tipo de servicio
yoy_volume_ticket_changes = (
    df.groupBy("year", "service_type")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_ticket"),
         F.round(F.sum("total_amount"), 2).alias("total_revenue"))
    .withColumn("prev_year_trips", F.lag("total_trips").over(F.Window.partitionBy("service_type").orderBy("year")))
    .withColumn("prev_year_ticket", F.lag("avg_ticket").over(F.Window.partitionBy("service_type").orderBy("year")))
    .withColumn("volume_change_pct", F.round(((F.col("total_trips") - F.col("prev_year_trips")) / F.col("prev_year_trips")) * 100, 2))
    .withColumn("ticket_change_pct", F.round(((F.col("avg_ticket") - F.col("prev_year_ticket")) / F.col("prev_year_ticket")) * 100, 2))
    .orderBy("service_type", "year")
)

print("CAMBIOS YOY EN VOLUMEN Y TICKET PROMEDIO POR SERVICE_TYPE:")
yoy_volume_ticket_changes.show(50)

In [38]:
# S. CAMBIOS YOY EN VOLUMEN Y TICKET PROMEDIO POR SERVICE_TYPE
print("ANÁLISIS S: Cambios YoY en Volumen y Ticket Promedio por Service_Type")

# Consulta SQL directa usando conector de Snowflake
query = """
WITH yearly_metrics AS (
    SELECT
        year,
        service_type,
        COUNT(*) as total_trips,
        ROUND(AVG(total_amount), 2) as avg_ticket,
        ROUND(SUM(total_amount), 2) as total_revenue
    FROM analytics.obt_trips
    WHERE service_type IS NOT NULL
    GROUP BY year, service_type
)
SELECT
    year,
    service_type,
    total_trips,
    avg_ticket,
    total_revenue,
    LAG(total_trips) OVER (PARTITION BY service_type ORDER BY year) as prev_year_trips,
    LAG(avg_ticket) OVER (PARTITION BY service_type ORDER BY year) as prev_year_ticket,
    ROUND(((total_trips - LAG(total_trips) OVER (PARTITION BY service_type ORDER BY year)) * 100.0 /
           LAG(total_trips) OVER (PARTITION BY service_type ORDER BY year)), 2) as volume_change_pct,
    ROUND(((avg_ticket - LAG(avg_ticket) OVER (PARTITION BY service_type ORDER BY year)) * 100.0 /
           LAG(avg_ticket) OVER (PARTITION BY service_type ORDER BY year)), 2) as ticket_change_pct
FROM yearly_metrics
ORDER BY service_type, year
LIMIT 50;
"""
execute_snowflake_sql(query)

ANÁLISIS S: Cambios YoY en Volumen y Ticket Promedio por Service_Type


([(2008, 'green', 89, 15.99, 1423.51, None, None, None, None),
  (2009, 'green', 273, 18.29, 4993.17, 89, 15.99, Decimal('206.74'), 14.38),
  (2010, 'green', 326, 18.36, 5984.06, 273, 18.29, Decimal('19.41'), 0.38),
  (2012, 'green', 3, 9.79, 29.36, 326, 18.36, Decimal('-99.08'), -46.68),
  (2015,
   'green',
   18936262,
   14.87,
   281511044.17,
   3,
   9.79,
   Decimal('631208633.33'),
   51.89),
  (2016,
   'green',
   16130618,
   14.69,
   236885515.02,
   18936262,
   14.87,
   Decimal('-14.82'),
   -1.21),
  (2017,
   'green',
   11571826,
   14.3,
   165486636.29,
   16130618,
   14.69,
   Decimal('-28.26'),
   -2.65),
  (2018,
   'green',
   8781384,
   16.2,
   142215050.78,
   11571826,
   14.3,
   Decimal('-24.11'),
   13.29),
  (2019,
   'green',
   5138221,
   18.53,
   95211274.17,
   8781384,
   16.2,
   Decimal('-41.49'),
   14.38),
  (2020,
   'green',
   1703900,
   20.34,
   34655123.67,
   5138221,
   18.53,
   Decimal('-66.84'),
   9.77),
  (2021,
   'green',
 

In [ ]:
# T. DÍAS CON ALTA CONGESTION_SURCHARGE - EFECTO EN TOTAL_AMOUNT VS DÍAS NORMALES
print("ANÁLISIS T: Días con Alta Congestion_Surcharge - Efecto en Total_Amount vs Días Normales")

# Análisis del efecto de la congestión en el total_amount comparando días con alta congestión vs días normales
congestion_effect_analysis = (
    df.withColumn("has_congestion", F.when(F.col("congestion_surcharge") > 0, "Con Congestión").otherwise("Sin Congestión"))
    .groupBy("pickup_date", "has_congestion")
    .agg(F.count("*").alias("total_trips"),
         F.round(F.avg("total_amount"), 2).alias("avg_total_amount"),
         F.round(F.avg("congestion_surcharge"), 2).alias("avg_congestion_surcharge"),
         F.round(F.avg("trip_duration_min"), 2).alias("avg_duration"),
         F.round(F.avg("avg_speed_mph"), 2).alias("avg_speed"))
    .withColumn("daily_trips", F.sum("total_trips").over(F.Window.partitionBy("pickup_date")))
    .withColumn("congestion_pct", F.round((F.col("total_trips") * 100.0 / F.col("daily_trips")), 2))
    .filter(F.col("has_congestion") == "Con Congestión")
    .withColumn("high_congestion_day", F.when(F.col("congestion_pct") >= 30, "Día Alta Congestión").otherwise("Día Normal"))
    .orderBy("pickup_date")
)

print("DÍAS CON ALTA CONGESTION_SURCHARGE - EFECTO EN TOTAL_AMOUNT VS DÍAS NORMALES:")
congestion_effect_analysis.show(50)

In [39]:
# T. DÍAS CON ALTA CONGESTION_SURCHARGE - EFECTO EN TOTAL_AMOUNT VS DÍAS NORMALES
print("ANÁLISIS T: Días con Alta Congestion_Surcharge - Efecto en Total_Amount vs Días Normales")

query = """
WITH daily_stats AS (
    SELECT
        pickup_date,
        CASE WHEN congestion_surcharge > 0 THEN 'Con Congestión' ELSE 'Sin Congestión' END AS has_congestion,
        COUNT(*) AS total_trips,
        ROUND(AVG(total_amount), 2) AS avg_total_amount,
        ROUND(AVG(congestion_surcharge), 2) AS avg_congestion_surcharge,
        ROUND(AVG(trip_duration_min), 2) AS avg_duration,
        ROUND(AVG(avg_speed_mph), 2) AS avg_speed
    FROM analytics.obt_trips
    WHERE pickup_date IS NOT NULL
    GROUP BY pickup_date, has_congestion
),
with_daily_totals AS (
    SELECT
        *,
        SUM(total_trips) OVER (PARTITION BY pickup_date) AS daily_trips,
        ROUND((total_trips * 100.0 / SUM(total_trips) OVER (PARTITION BY pickup_date)), 2) AS congestion_pct
    FROM daily_stats
)
SELECT
    pickup_date,
    has_congestion,
    total_trips,
    avg_total_amount,
    avg_congestion_surcharge,
    avg_duration,
    avg_speed,
    daily_trips,
    congestion_pct,
    CASE WHEN congestion_pct >= 30 THEN 'Día Alta Congestión' ELSE 'Día Normal' END AS high_congestion_day
FROM with_daily_totals
WHERE has_congestion = 'Con Congestión'
ORDER BY pickup_date
LIMIT 50;
"""
execute_snowflake_sql(query)

ANÁLISIS T: Días con Alta Congestion_Surcharge - Efecto en Total_Amount vs Días Normales


([(datetime.date(2001, 1, 1),
   'Con Congestión',
   12,
   41.63,
   2.5,
   467.67,
   5.57,
   20,
   Decimal('60.00'),
   'Día Alta Congestión'),
  (datetime.date(2002, 2, 2),
   'Con Congestión',
   11,
   17.9,
   2.5,
   241.73,
   7.65,
   11,
   Decimal('100.00'),
   'Día Alta Congestión'),
  (datetime.date(2002, 10, 21),
   'Con Congestión',
   17,
   19.01,
   2.5,
   11.65,
   13.66,
   23,
   Decimal('73.91'),
   'Día Alta Congestión'),
  (datetime.date(2002, 10, 22),
   'Con Congestión',
   36,
   18.12,
   2.5,
   13.22,
   8.84,
   43,
   Decimal('83.72'),
   'Día Alta Congestión'),
  (datetime.date(2002, 10, 23),
   'Con Congestión',
   25,
   15.15,
   2.5,
   10.32,
   11.9,
   38,
   Decimal('65.79'),
   'Día Alta Congestión'),
  (datetime.date(2002, 10, 24),
   'Con Congestión',
   24,
   18.66,
   2.5,
   12.67,
   10.64,
   31,
   Decimal('77.42'),
   'Día Alta Congestión'),
  (datetime.date(2002, 10, 25),
   'Con Congestión',
   25,
   19.38,
   2.5,
   12.08,
